In [1]:
import os
import cv2
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from keras import Input

In [2]:
resume_folder = '/content/drive/MyDrive/resumes'
non_resume_folder = '/content/drive/MyDrive/not_resumes'

In [3]:
# Load and Explore the Data
def load_and_preprocess_data(data_dir, label, augmentation):
    data = []
    labels = []
    for filename in os.listdir(data_dir):
        img_path = os.path.join(data_dir, filename)

        # Check if the file exists and is a valid image
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)

            # Check if the image is successfully loaded
            if img is not None:
                img = cv2.resize(img, (128, 128))
                img = img / 255.0
                data.append(img)
                labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    if augmentation:
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.1,
            shear_range=0.2,
            horizontal_flip=True
        )
        datagen.fit(data)
        augmented_data = datagen.flow(data, labels, batch_size=len(data), shuffle=False).next()
        data, labels = augmented_data[0], labels

    return data, labels

In [4]:
# Load and preprocess data from the two folders
resume_data_aug, resume_labels_aug = load_and_preprocess_data(resume_folder, 1, True)
non_resume_data_aug, non_resume_labels_aug = load_and_preprocess_data(non_resume_folder, 0, True)

resume_data, resume_labels = load_and_preprocess_data(resume_folder, 1, False)
non_resume_data, non_resume_labels = load_and_preprocess_data(non_resume_folder, 0, False)

In [5]:
# Combine data and labels
X_train_aug = np.concatenate((resume_data_aug, non_resume_data_aug), axis=0)
y_train_aug = np.concatenate((resume_labels_aug, non_resume_labels_aug), axis=0)

X_train = np.concatenate((resume_data, non_resume_data), axis=0)
y_train = np.concatenate((resume_labels, non_resume_labels), axis=0)

In [6]:
# Shuffle the data
augmented_data = list(zip(X_train_aug, y_train_aug))
np.random.shuffle(augmented_data)
X_train_aug, y_train_aug = zip(*augmented_data)
X_train_aug = np.array(X_train_aug)
y_train_aug = np.array(y_train_aug)

In [7]:
# Split the data into training and testing sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train_aug, y_train_aug, test_size=0.2, random_state=42 , stratify=y_train_aug
)

In [8]:
import pandas as pd

# Assuming 'y_train' is your training labels
class_distribution = pd.Series(y_train.flatten()).value_counts()
print("Class Distribution:")
print(class_distribution)

Class Distribution:
1    117
0    117
dtype: int64


In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(128, 128, 3)))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

9406464/9406464 [==============================] - 0s 0us/step


In [10]:
# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
epochs = 10
batch_size = 32

history = model.fit(X_train_split, y_train_split, batch_size=batch_size, epochs=epochs, validation_data=(X_test_split, y_test_split))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_split, y_test_split, batch_size=batch_size)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
6/6 [==============================] - 9s 805ms/step - loss: 0.4707 - accuracy: 0.7807 - val_loss: 0.4147 - val_accuracy: 0.8085
Epoch 2/10
6/6 [==============================] - 2s 433ms/step - loss: 0.2073 - accuracy: 0.9091 - val_loss: 0.2772 - val_accuracy: 0.8723
Epoch 3/10
6/6 [==============================] - 2s 429ms/step - loss: 0.1131 - accuracy: 0.9733 - val_loss: 0.2324 - val_accuracy: 0.8936
Epoch 4/10
6/6 [==============================] - 2s 432ms/step - loss: 0.0746 - accuracy: 0.9679 - val_loss: 0.2242 - val_accuracy: 0.9149
Epoch 5/10
6/6 [==============================] - 4s 808ms/step - loss: 0.0463 - accuracy: 0.9947 - val_loss: 0.2095 - val_accuracy: 0.9149
Epoch 6/10
6/6 [==============================] - 3s 512ms/step - loss: 0.0372 - accuracy: 1.0000 - val_loss: 0.2268 - val_accuracy: 0.9149
Epoch 7/10
6/6 [==============================] - 2s 390ms/step - loss: 0.0352 - accuracy: 1.0000 - val_loss: 0.2365 - val_accuracy: 0.9149
Epoch 8/10
6/6 [====

In [12]:
from sklearn.metrics import log_loss, cohen_kappa_score, roc_auc_score

# Evaluate the model on the test set
y_pred = model.predict(X_test_split)

# Thresholds
thresholds = [0.2, 0.3, 0.4, 0.5, 0.6]

for threshold in thresholds:
    y_pred_binary = (y_pred > threshold).astype(int)

    # Calculate metrics
    precision = precision_score(y_test_split, y_pred_binary)
    recall = recall_score(y_test_split, y_pred_binary)
    f1 = f1_score(y_test_split, y_pred_binary)
    logloss = log_loss(y_test_split, y_pred_binary)
    kappa = cohen_kappa_score(y_test_split, y_pred_binary)

    # ROC-AUC requires binary or multilabel indicator format
    roc_auc = roc_auc_score(y_test_split, y_pred_binary)

    print(f"Threshold: {threshold}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Log Loss: {logloss:.4f}, Cohen's Kappa: {kappa:.4f}, ROC-AUC: {roc_auc:.4f}")

# Choose an appropriate threshold, e.g., 0.5
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate metrics
precision = precision_score(y_test_split, y_pred_binary)
recall = recall_score(y_test_split, y_pred_binary)
f1 = f1_score(y_test_split, y_pred_binary)
logloss = log_loss(y_test_split, y_pred_binary)
kappa = cohen_kappa_score(y_test_split, y_pred_binary)
roc_auc = roc_auc_score(y_test_split, y_pred_binary)

# Confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred_binary)

print("\nOverall Metrics:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Log Loss:", logloss)
print("Cohen's Kappa:", kappa)
print("ROC-AUC:", roc_auc)
print("\nConfusion Matrix:")
print(conf_matrix)


2/2 [==============================] - 2s 216ms/step
Threshold: 0.2, Precision: 0.8148, Recall: 0.9565, F1: 0.8800, Log Loss: 4.6013, Cohen's Kappa: 0.7455, ROC-AUC: 0.8741
Threshold: 0.3, Precision: 0.8462, Recall: 0.9565, F1: 0.8980, Log Loss: 3.8344, Cohen's Kappa: 0.7877, ROC-AUC: 0.8949
Threshold: 0.4, Precision: 0.8462, Recall: 0.9565, F1: 0.8980, Log Loss: 3.8344, Cohen's Kappa: 0.7877, ROC-AUC: 0.8949
Threshold: 0.5, Precision: 0.8800, Recall: 0.9565, F1: 0.9167, Log Loss: 3.0675, Cohen's Kappa: 0.8300, ROC-AUC: 0.9158
Threshold: 0.6, Precision: 0.9167, Recall: 0.9565, F1: 0.9362, Log Loss: 2.3007, Cohen's Kappa: 0.8724, ROC-AUC: 0.9366

Overall Metrics:
Precision: 0.88
Recall: 0.9565217391304348
F1 Score: 0.9166666666666666
Log Loss: 3.0675449692865664
Cohen's Kappa: 0.8300180831826401
ROC-AUC: 0.9157608695652174

Confusion Matrix:
[[21  3]
 [ 1 22]]
